<a href="https://colab.research.google.com/github/salomao2005/Cookbook/blob/master/dia4_imers%C3%A3o_chatbots_whatsapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conectando ao Web Whatsapp - Parte 1
---
No BOOTCAMP CHATBOTS PARA WHATSAPP nos aprofundamos nas features e capacidades do chatbot(como envio de arquivos/imagens), tanto em API´s e Webhooks, como na escalabilidade/disponibilidade na conexão junto ao Whatsapp Mobile.

In [0]:
# IMPORTAR AS LIBS
import os
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
from chatterbot.trainers import ListTrainer

In [0]:
#INSTANCIAR CHATBOT
chatbot = ChatBot('Ananda')
trainer = ChatterBotCorpusTrainer(chatbot)
trainer.train('chatterbot.corpus.portuguese')
trainerer = ListTrainer(chatbot)

# ARMAZENAR DIRETORIO PRINCIPAL EM VARIAVEL
dir_path = os.getcwd()

In [0]:
# INICIAR APLICAÇÃO
driver = webdriver.Chrome(dir_path+'/chromedriver.exe') 
driver.get('https://web.whatsapp.com/')
driver.implicitly_wait(15)

In [0]:
# FUNÇÕES BÁSICAS DE COMUNICAÇÃO
def pegaConversa():
	try:
		post = driver.find_elements_by_class_name("_12pGw")
		ultimo = len(post) - 1
		texto = post[ultimo].find_elements_by_css_selector("span.selectable-text").text
		return texto
	except:
		pass

def enviaMensagem(mensagem):
	caixa_de_texto = driver.find_element_by_class_name('_3u328')
	valor = "*Ananda:* "+str(mensagem)
	for part in valor.split("\n"):
		caixa_de_texto.send_keys(part)
		ActionChains(driver).key_down(keys.SHIFT).key_down(keys.ENTER).key_up(keys.SHIFT).perform()
	time.sleep(0.5)
	botao_enviar = driver.find_elements_by_class_name('_3M-N-')
	botao_enviar.click()

def treinar(mensagem):
	resposta = 'Como respondo isso? me ensina, por favor...? utilize ;"'+str(mensagem)+'"'
	enviaMensagem(resposta)
	novo = []
	try:
		while True:
			ultima = pegaConversa()
			if ultima == "!":
				enviaMensagem("Você desativou meu aprendizado.")
				break
			elif ultima.replace(';','') != '' and ultima != mensagem and ultima[0] == ";" :
				auxiliar = ultima
				print(mensagem.lower().strip())
				print(ultima.replace(';','').lower().strip())
				novo.append(mensagem.lower().strip())
				novo.append(ultima.replace(';','').lower().strip())
				trainerer.train(novo)
				enviaMensagem("Pronto, aprendi! Obrigada <3")
				break
	except:
		pass